## Loan Approval Prediction & Credit Risk Analysis


### Libraries

In [ ]:
import pandas as pd
import numpy as np
impot matplotlib.pyplot as plt
import seaborn as sns
import sickit-learn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
